In [ ]:
!sudo apt-get install git-lfs
!git lfs install

In [19]:
!git clone https://huggingface.co/blanchefort/rubert-base-cased-sentiment

Cloning into 'rubert-base-cased-sentiment'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 5), reused 0 (delta 0)
Unpacking objects: 100% (21/21), done.
Filtering content: 100% (3/3), 1.98 GiB | 88.16 MiB/s, done.


In [20]:
from graph import InferenceWorker
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('/content/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('/content/rubert-base-cased-sentiment', return_dict=True)

In [21]:
text = 'В стране карантин. Умерло очень много людей.'

In [22]:
class SampleEmotion(InferenceWorker):
    def __call__(self, t, d):
        if t == "sum_text":
            yield "emotion", self.predict(d), "Emotion_detection"
        else:
            yield None        
    @torch.no_grad()
    def predict(self, text):
        inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**inputs)
        predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted = torch.argmax(predicted, dim=1).numpy()
        if predicted[0] == 0:
          return 'NEUTRAL'
        if predicted[0] == 1:
          return 'POSITIVE'
        if predicted[0] == 2:
          return 'NEGATIVE'     

In [23]:
worker = SampleEmotion()
worker.serialize()

Worker saved!


In [24]:
for x in worker("sum_text", text):
  print(x)

('emotion', 'NEGATIVE', 'Emotion_detection')
